In [77]:
from fontTools.varLib.plot import stops
!pip install requests beautifulsoup4 pandas selenium

import pandas as pd
import requests
from bs4 import BeautifulSoup


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 7.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


# Step 1 get list of project links

In [82]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

options = Options()
options.add_argument("--headless")  # don't open a browser window

driver = webdriver.Chrome(options=options)
driver.get("https://www.projets-recherche.ulaval.ca/en/")
print(driver)


<selenium.webdriver.chrome.webdriver.WebDriver (session="3e34a94ff0318943d83cca7a4da7e9b4")>


In [83]:

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")


In [105]:
#headers = {"User-Agent": "Mozilla/5.0"}
#response = requests.get(list_url, headers=headers)
#print("Status Code:", response.status_code)
#print("First 500 characters:\n", response.text[:500])


#soup = BeautifulSoup(response.text, "html.parser")


Status Code: 200


In [106]:
with open("debug_page.html", "w", encoding="utf-8") as f:
    f.write(soup.text)


In [106]:
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
import time

# Helper to grab everything under a given <h2> until the next <h2>
def get_section_text(soup, heading_name):
    h2 = soup.find("h2", string=heading_name)
    if not h2:
        return None

    content_parts = []
    for sibling in h2.next_siblings:
        if sibling.name == "h2":  # stop at the next section
            break
        if sibling.name == "br":
            content_parts.append(" ")  # treat <br> as a space
        elif isinstance(sibling, NavigableString):
            text = sibling.strip()
            if text:
                content_parts.append(text)
        elif hasattr(sibling, "get_text"):
            text = sibling.get_text(strip=True)
            if text:
                content_parts.append(text)

    return " ".join(content_parts)

data = []
rows = soup.find_all("tr")

for row in rows[1:]:                     # skip header
    cols = row.find_all("td")
    if len(cols) < 3:
        continue

    # List‐page fields
    project_id       = cols[0].text.strip()
    opportunity_type = cols[1].text.strip()
    content          = cols[2]

    # Safe name split
    prof_tag  = content.find("em", class_="chercheur")
    full_name = prof_tag.text.strip() if prof_tag else ""
    if full_name:
        parts = full_name.split()
        first = parts[0]
        last  = " ".join(parts[1:]) if len(parts) > 1 else ""
    else:
        first, last = "", ""

    title_tag = content.find("a")
    title     = title_tag.text.strip() if title_tag else ""
    env_tag   = content.find("span", class_="milieu")
    environment = env_tag.text.strip() if env_tag else ""

    full_url = f"https://www.projets-recherche.ulaval.ca/en/project?id={project_id}"

    # Fetch and parse detail page
    driver.get(full_url)
    time.sleep(1)
    soup_detail = BeautifulSoup(driver.page_source, "html.parser")

    # Email
    email_tag = soup_detail.find("a", href=lambda x: x and x.startswith("mailto:"))
    email     = email_tag.text.strip() if email_tag else ""

    # Financial aid
    aid_h4      = soup_detail.find(
                      "h4",
                      string="Financial Aid Related to Research Project"
                  )
    if aid_h4:
        p = aid_h4.find_next_sibling("p")
        financial_aid = p.text.strip() if p else ""
    else:
        financial_aid = ""

    # Combine all fields into one row
    row_data = {
        "project_id":           project_id,
        "title":                title,
        "opportunity_type":     opportunity_type,
        "professor_first_name": first,
        "professor_last_name":  last,
        "environment":          environment,
        "page_url":             full_url,
        "description":          get_section_text(soup_detail, "Description"),
        "research_field":       get_section_text(soup_detail, "Research Field"),
        "research_supervisors": get_section_text(soup_detail, "Research Supervisors"),
        "research_environment": get_section_text(soup_detail, "Research Environment"),
        "desired_profile":      get_section_text(soup_detail, "Desired Profile"),
        "email":                email,
        "financial_aid":        financial_aid
    }

    data.append(row_data)

# Build DataFrame once, after the loop
df = pd.DataFrame(data)
print(df)



   project_id                                              title  \
0      347768  Development of wood innovative hardening solut...   
1      357687  Roles of matricryptins (bioactive extracellula...   
2      392562  The neuroplasticity of the subthalamic nucleus...   
3      442396                   Wound healing of irradiated skin   
4      432421  Nanomedicines: what is the fate of nanoparticl...   
5      331975  Migration, Security and Resilience: Comparativ...   
6      479868  Abnormal cell division in the development of c...   
7      457680  Funded PhD on crop-associated phyllosphere mic...   
8      441685  Health determinants in a population receiving ...   
9      483209  Formulation of filtration membrane with contro...   
10     479359  Studying the effect of an innovative and non-i...   
11     326267           Assistive technologies in rehabilitation   
12     244091     Key players in insulin resistance and diabetes   
13     365153  Application of Machine Learning A

In [107]:
df

,project_id,title,opportunity_type,professor_first_name,professor_last_name,environment,page_url,description,research_field,research_supervisors,research_environment,desired_profile,email,financial_aid
0,347768,Development of wood innovative hardening solut...,Master’s,Véronic,Landry,NSERC/Canlak Industrial Research Chair in inte...,https://www.projets-recherche.ulaval.ca/en/pro...,The objective of chemical wood densification i...,- Chemistry - Chemical engineering - Wood ...,Véronic Landry Stephanie Vanslambrouck,NSERC/Canlak Industrial Research Chair in inte...,- Wood Engineering Requirements and Conditions...,veronic.landry@sbf.ulaval.ca,$17000 per year for 2 years.
1,357687,Roles of matricryptins (bioactive extracellula...,Master’s Ph.D.,Julie,Fradette,Centre de recherche en organogénèse expériment...,https://www.projets-recherche.ulaval.ca/en/pro...,The laboratory of Julie Fradette is seeking a ...,- Tissue engineering - Stem cells - 3D mod...,None,Centre de recherche en organogénèse expériment...,- Medecine - Pharmacy - Biomedical Science...,julie.fradette@fmed.ulaval.ca,$21000 per year for 3 years.
2,392562,The neuroplasticity of the subthalamic nucleus...,Master’s Ph.D.,Martin,Parent,CERVO Brain Research Centre,https://www.projets-recherche.ulaval.ca/en/pro...,The goal of this research project is to charac...,- Neurosciences - Parkinson' disease - Mon...,None,CERVO Brain Research Centre CERVO Research Cen...,- Biomedical Science - Neurology - Biochem...,martin.parent@fmed.ulaval.ca,$21500 per year for 4 years.
3,442396,Wound healing of irradiated skin,Master’s Ph.D.,Julie,Fradette,Centre de recherche en organogénèse expériment...,https://www.projets-recherche.ulaval.ca/en/pro...,The laboratory of Julie Fradette is seeking a ...,- Tissue engineering - Stem cells - 3D mod...,None,Centre de recherche en organogénèse expériment...,- Medecine - Pharmacy - Biomedical Science...,julie.fradette@fmed.ulaval.ca,$22000 per year for 4 years.
4,432421,Nanomedicines: what is the fate of nanoparticl...,Master’s Ph.D.,Nicolas,Bertrand,CHU de Quebec - Université Laval Research Center,https://www.projets-recherche.ulaval.ca/en/pro...,This multidisciplinary project is at the inter...,- Pharmaceutical sciences - Nanomedicine -...,None,CHU de Quebec - Université Laval Research Cent...,- Medecine - Pharmacy - Nursing - Biomed...,nicolas.bertrand@pha.ulaval.ca,$22000 per year for 2 years.
5,331975,"Migration, Security and Resilience: Comparativ...",Master’s Ph.D.,,,Graduate School of International Studies,https://www.projets-recherche.ulaval.ca/en/pro...,This project wants to better understand the li...,- International relation - Security - Migr...,None,Graduate School of International Studies Web Site,"- Administration, Organizational Information S...",philippe.bourbeau@pol.ulaval.ca,Information unavailable
6,479868,Abnormal cell division in the development of c...,Master’s Ph.D.,Sabine,Elowe,CHU de Québec Research Centre (CRCHU) (CR-NCH ...,https://www.projets-recherche.ulaval.ca/en/pro...,To better understand the role of aberrant mito...,- Cell Biology - Molecular Biology - Mitos...,None,CHU de Québec Research Centre (CRCHU) (CR-NCH ...,- Biomedical Science - Biochemistry - Biol...,sabine.elowe@crchuq.ulaval.ca,$25000 per year for 4 years.
7,457680,Funded PhD on crop-associated phyllosphere mic...,Ph.D.,Jean-Baptiste,Leducq,Centre de recherche et d'innovation sur les vé...,https://www.projets-recherche.ulaval.ca/en/pro...,"Leducq Lab (Phytology Department, IBIS, CRIV, ...",- Plant microbiome - Microbial genomics - ...,None,Centre de recherche et d'innovation sur les vé...,- Agronomy - Biology Requirements and Condit...,JBLED@ulaval.ca,Information unavailable
8,441685,Health determinants in a population receiving ...,Master’s,Aimée,Dawson,"Faculty of dentistry, research center the GREB",https://www.projets-recherche.ulaval.ca/en/pro...,The aim of this project is to conduct a clinic...,- Health determinants - Dental health - Co...,Aimée Daw

# Cleaning data into website format

In [111]:
import pandas as pd


FinalData = pd.DataFrame({
    'Title': df['title'],
    'Description': df['description'],
    'General Ressearch Area': df['research_field'].str.replace('   - ', ', ').str.replace('- ', ''),
    'Application deadline': 'NA',
    'Currency': 'CAD',
    'Funding per Year': df['financial_aid'],
    'Funding expectation': 'Available upon request',
    'Languages': 'French or English',
    'Skills': df['desired_profile'].str.replace("   -\xa0", ", ").str.replace("-\xa0", "").str.replace("- ", ", "),
    'City': 'Quebec City',
    'Province': 'Quebec',
    'Country': 'Canada',
    'Additional Questions': '',
    'professor_first_name': df['professor_first_name'],
    'professor_last_name': df['professor_last_name'],
    'University': 'Université Laval',
    'Faculty': '',
    'Department': '',
    'Email': df['email']

})


FinalData



,Title,Description,General Ressearch Area,Application deadline,Currency,Funding per Year,Funding expectation,Languages,Skills,City,Province,Country,Additional Questions,professor_first_name,professor_last_name,University,Faculty,Department,Email
0,Development of wood innovative hardening solut...,The objective of chemical wood densification i...,"Chemistry, Chemical engineering, Wood engineer...",NA,CAD,$17000 per year for 2 years.,Available upon request,French or English,"Wood Engineering Requirements and Conditions, ...",Quebec City,Quebec,Canada,,Véronic,Landry,Université Laval,,,veronic.landry@sbf.ulaval.ca
1,Roles of matricryptins (bioactive extracellula...,The laboratory of Julie Fradette is seeking a ...,"Tissue engineering, Stem cells, 3D models, Ang...",NA,CAD,$21000 per year for 3 years.,Available upon request,French or English,"Medecine, Pharmacy, Biomedical Science, Bioche...",Quebec City,Quebec,Canada,,Julie,Fradette,Université Laval,,,julie.fradette@fmed.ulaval.ca
2,The neuroplasticity of the subthalamic nucleus...,The goal of this research project is to charac...,"Neurosciences, Parkinson' disease, Monkey, Neu...",NA,CAD,$21500 per year for 4 years.,Available upon request,French or English,"Biomedical Science, Neurology, Biochemistry, B...",Quebec City,Quebec,Canada,,Martin,Parent,Université Laval,,,martin.parent@fmed.ulaval.ca
3,Wound healing of irradiated skin,The laboratory of Julie Fradette is seeking a ...,"Tissue engineering, Stem cells, 3D models, Wou...",NA,CAD,$22000 per year for 4 years.,Available upon request,French or English,"Medecine, Pharmacy, Biomedical Science, Bioche...",Quebec City,Quebec,Canada,,Julie,Fradette,Université Laval,,,julie.fradette@fmed.ulaval.ca
4,Nanomedicines: what is the fate of nanoparticl...,This multidisciplinary project is at the inter...,"Pharmaceutical sciences, Nanomedicine, Biochem...",NA,CAD,$22000 per year for 2 years.,Available upon request,French or English,"Medecine, Pharmacy, Nursing, Biomedical Scienc...",Quebec City,Quebec,Canada,,Nicolas,Bertrand,Université Laval,,,nicolas.bertrand@pha.ulaval.ca
5,"Migration, Security and Resilience: Comparativ...",This project wants to better understand the li...,"International relation, Security, Migration, R...",NA,CAD,Information unavailable,Available upon request,French or English,"Administration, Organizational Information Sys...",Quebec City,Quebec,Canada,,,,Université Laval,,,philippe.bourbeau@pol.ulaval.ca
6,Abnormal cell division in the development of c...,To better understand the role of aberrant mito...,"Cell Biology, Molecular Biology, Mitosis, Cell...",NA,CAD,$25000 per year for 4 years.,Available upon request,French or English,"Biomedical Science, Biochemistry, Biology, Mic...",Quebec City,Quebec,Canada,,Sabine,Elowe,Université Laval,,,sabine.elowe@crchuq.ulaval.ca
7,Funded PhD on crop-associated phyllosphere mic...,"Leducq Lab (Phytology Department, IBIS, CRIV, ...","Plant microbiome, Microbial genomics, Metageno...",NA,CAD,Information unavailable,Available upon request,French or English,"Agronomy, Biology Requirements and Conditions,...",Quebec City,Quebec,Canada,,Jean-Baptiste,Leducq,Université Laval,,,JBLED@ulaval.ca
8,Health determinants in a population receiving ...,The aim of this project is to conduct a clinic...,"Health determinants, Dental health, Community ...",NA,CAD,$15000 per year for 2 years.,Available upon request,French or English,"Supply chain, operations and logistics, Dental...",Quebec City,Quebec,Canada,,Aimée,Dawson,Université Laval,,,Aimee.dawson@fmd.ulaval.ca
9,Formulation of filtration membrane with contro...,The use of different membranes in the food ind...,"Electrodialysis, Filtration membrane, Zeta pot...",NA,CAD,$22000 per year for 3 years.,Available upon request,French or English,Food Science and Technology Requirements and C...,Quebec City,Quebec,Canada,,Laurent,Bazinet,Université Laval,,,veronique.perreault.5@ulaval.ca
